# Assignment 5 - Reinforcement Learning

### RBOT240 - Machine Learning

#### Instructor: Daniel Pineo &copy; 2021

#### Submitted: Salinee Kingbaisomboon

# FrozenLake

In this assignment we're going to be looking at the FrozenLake problem in the OpenAI Gym simulation envronment, described below. 

![image](https://drive.google.com/uc?export=view&id=1ExLANuLFc4LW8g84_1gKJj4gClvQpEkW)

The environment has 16 states and 4 actions:

![image](https://drive.google.com/uc?export=view&id=1Y6B_jH9eNibXL_xMHMEMl2ID1tvj_Hnx)

Documentation for the OpenAI FrozenLake environment can be found [here](https://gym.openai.com/envs/FrozenLake-v0/).

We'll be using this environment to experiment with our first reinforcement algorithms: Policy Iteration and Value Iteration.

In [1]:
import gym
import numpy as np
import torch

# The grid can be 4x4 or 8x8, but we'll start with the 4x4 for simplicity.
grid_dim = 4     # can be 4 or 8
map_name = f"{grid_dim}x{grid_dim}"

# Gamma is the discount rate for future rewards
gamma = 0.9

# Setting is_slippery to True results in a 33% chance to move 
# in a direction 90 degrees from the intended direction when
# taking an action.
env = gym.make("FrozenLake-v0", map_name=map_name, is_slippery=False).env
env.render()


SFFF
FHFH
FFFH
HFFG


In [2]:
# Helper function to visualize a policy.  A policy is a probability 
# distribution over possible actions for each state.  However, we will
# only show the action associated with the highest-value for each state.

def draw_policy(policy):
    action_symbols = "<v>^"
    best_policy = np.argmax(policy, axis=-1).reshape([-1, grid_dim])
    policy_arrows = np.vectorize(lambda x: action_symbols[x])(best_policy)
    print(np.where(env.desc == b'F', policy_arrows, env.desc))

# Policy Iteration

In this section, we'll be implementing the policy iteration algorithm (Sutton & Barto, section 4.3).  One of the steps of this algorithm that we'll be implementing is the policy evaluation, in which the value function is calculated for a given policy.

![image](https://drive.google.com/thumbnail?id=113NOzgN9o9ITaW3t0wHENT0Sb1pXUTzg&sz=w800)

In [3]:
def policy_evaluation(env, policy, gamma=1.0):
    state_value = torch.zeros(env.nS)  # state-value function.  initialize arbitrarily
    theta=1e-8
    while True:
        state_value_previous = state_value.detach().clone()

        # Calculate state_value, the state-value function according to the formula 
        # above.  It should be implemented as an array of shape (num_states,)

        # hint: generally like to do use vector/matrix operations when I can, 
        # but the OpenAI environment doesn't have the data stored that way, so
        # I used a bunch of nested loops to accumulate over the sums

        # hint: some of the information you need is in env:
        #   env.nS = number of states
        #   env.nA = number of actions
        #   env.P[state][action] = (transition_probability, next_state, reward, is_done)

        ################################################
        ####### Place your implementation here #########
        ################################################
        delta = 0
        for s in range(env.nS):
          v = 0
          for a, action_prob in enumerate(policy[s]):
            for prob, next_state, reward, done in env.P[s][a]:
              v += action_prob * prob * (reward + gamma * state_value[next_state])
          delta = max(delta, abs(v-state_value[s]))
          state_value[s] = v
        if delta < theta:
          break

        if torch.allclose(state_value_previous, state_value): 
            break

    return state_value

# Test the random policy (25% chance to go in each direction)
assert torch.allclose(
    policy_evaluation(env, torch.ones([env.nS, env.nA]) / env.nA), 
    torch.tensor([
            0.0139398 , 0.01163093, 0.02095299, 0.01047649,
            0.01624867, 0.        , 0.04075154, 0.        ,
            0.0348062 , 0.08816993, 0.14205316, 0.        ,
            0.        , 0.17582037, 0.43929118, 0.        
        ])
    , atol=1e-6)

Next we'll look at how the policy evaluation is used in the subsequent policy improvement step.

![image](https://drive.google.com/thumbnail?id=1p0q0MI95JZyg_eLonJJZL9mlP1Dd0kEg&sz=w800)

In [4]:
def policy_improvement(env, state_value, gamma, debug=False):

    # Calculate action_value, the action-value function according to the formula above.
    # It should be implemented as an array of shape (num_states, num_actions)

    # (hint: same hints as above)

    ################################################
    ####### Place your implementation here #########
    ################################################

    # Initialize variables
    policy = torch.zeros([env.nS, env.nA])
    action_value = torch.zeros([env.nS, env.nA])

    for s in range(env.nS):
      for a, action_prob in enumerate(policy[s]):
            for prob, next_state, reward, done in env.P[s][a]:
              action_value[s][a] += prob * (reward + gamma * state_value[next_state])

    if debug:
        assert torch.allclose(
            action_value,
            torch.tensor([
                [0.4466, 0.2767, 0.6914, 0.4466],
                [0.4466, 0.5707, 0.0796, 0.6914],
                [0.6914, 0.4411, 0.1188, 0.0796],
                [0.0796, 0.8068, 0.1188, 0.1188],
                [0.2767, 0.4101, 0.5707, 0.4466],
                [0.5707, 0.5707, 0.5707, 0.5707],
                [0.5707, 0.3140, 0.8068, 0.0796],
                [0.8068, 0.8068, 0.8068, 0.8068],
                [0.4101, 0.0201, 0.5691, 0.2767],
                [0.4101, 0.1520, 0.3140, 0.5707],
                [0.5691, 0.2645, 0.3615, 0.4411],
                [0.3615, 0.3615, 0.3615, 0.3615],
                [0.0201, 0.0201, 0.0201, 0.0201],
                [0.0201, 0.1520, 0.2645, 0.5691],
                [0.1520, 0.2645, 1.4667, 0.3140],
                [0.4667, 0.4667, 0.4667, 0.4667]
            ]), atol=1e-4
        )


    # Calculate policy, the greed policy by performing the argmax over the 
    # action_value function.  Implement the policy function
    # as a matrix of size (num_states, num_actions).  
    
    ################################################
    ####### Place your implementation here #########
    ################################################
    policy_stable = True

    for s in range(env.nS):
      old_action = policy[s].detach().clone()
      
      max_index = torch.argmax(action_value[s]).data # find index of max action's value
      policy[s][max_index] = 1 # update policy

      if torch.allclose(old_action,policy[s]) == False: 
        policy_stable = False
      if policy_stable: 
        break

    if debug:
        assert torch.allclose(
            policy,
            torch.tensor([
              [0., 0., 1., 0.],
              [0., 0., 0., 1.],
              [1., 0., 0., 0.],
              [0., 1., 0., 0.],
              [0., 0., 1., 0.],
              [1., 0., 0., 0.],
              [0., 0., 1., 0.],
              [1., 0., 0., 0.],
              [0., 0., 1., 0.],
              [0., 0., 0., 1.],
              [1., 0., 0., 0.],
              [1., 0., 0., 0.],
              [1., 0., 0., 0.],
              [0., 0., 0., 1.],
              [0., 0., 1., 0.],
              [1., 0., 0., 0.]
            ])
        )

    return policy

torch.manual_seed(0)
test = policy_improvement(env, torch.rand(env.nS), .9, True)

Now that we've implemente all the pieces, we put them together to form the full algorithm.

In [5]:
policy = torch.ones([env.nS, env.nA]) / env.nA  # initialize arbitrary: 25% each action
state_value = torch.zeros(env.nS)  # state-value function.  initialize arbitrarily

for n in range(10):
    # calculate new value function from the current policy    
    state_value = policy_evaluation(env, policy, gamma)

    # calculate new policy from the value function
    policy = policy_improvement(env, state_value, gamma)
draw_policy(policy)

[['S' '>' 'v' '<']
 ['v' 'H' 'v' 'H']
 ['>' 'v' 'v' 'H']
 ['H' '>' '>' 'G']]


# Value Iteration

Value iteration is quite similar to policy iteration, but we only do a single iteration on the policy evaluation.

![image](https://drive.google.com/thumbnail?id=15HoyTTKbFLS58uyeZllUuopCu1JJfJry&sz=w800)


In [6]:
policy = torch.ones([env.nS, env.nA]) / env.nA
state_value = torch.zeros([env.nS])
theta=1e-8

def action_value_one_iter_f(env, state_value, s, gamma=1):
    action_value = torch.zeros(env.nA)
    for a in range(env.nA):
        for prob, next_state, reward, done in env.P[s][a]:
            action_value[a] += prob * (reward + gamma * state_value[next_state])
    return action_value

while True:
  state_value_previous = state_value.detach().clone()

  # calculate state_value, the state-value function update iteration above.  

  # (hint: you've already implemented the action_value calculation)

  ################################################
  ####### Place your implementation here #########
  ################################################
  delta = 0
  for s in range(env.nS):
    v = state_value[s].detach().clone()
    state_value[s] = max(action_value_one_iter_f(env, state_value, s, gamma))
    delta = max(delta,abs(v-state_value[s]))

  if delta < theta:
    break

  if not state_value_previous.sum():
      assert torch.allclose(
          state_value,
          torch.tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
          atol=1e-4
      )

  if torch.allclose(state_value_previous, state_value): 
      break

# Calculate policy, the optimal deterministic policy from the action_value 
# function as described above

# ################################################
# ####### Place your implementation here #########
# ################################################
policy = policy_improvement(env, state_value, gamma)

assert torch.allclose(
    policy,
    torch.tensor([
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.]
      ]),
    atol=1e-4
)

draw_policy(policy)

[['S' '>' 'v' '<']
 ['v' 'H' 'v' 'H']
 ['>' 'v' 'v' 'H']
 ['H' '>' '>' 'G']]
